In [28]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.utils import np_utils, plot_model
from sys import stdout

In [29]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('font', family = 'serif', size = 17)
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.minor.size'] = 2
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.minor.size'] = 2

### Getting data sets

In [30]:
fp = open('data/train.txt','r')
raw_x = []
raw_y = []
for line in fp:
    tmp = list(map(float, line.split(' ')))
    a = tmp[-1]
    raw_x.append(np.array(tmp[:-1] + [0., 0., 0.]))
    if(a > 0.5):
        raw_y.append(2)
    elif(a < -0.5):
        raw_y.append(1)
    else:
        raw_y.append(0)
fp.close()

In [31]:
def getMat(arr, win):
    ans = []
    N = len(arr) - win + 1
    batch_size = N/50
    for i in range(len(arr) - win + 1):
        if(i%batch_size == 0):
            stdout.write('=')
            stdout.flush()
        ans.append(np.array([np.array(vec) for vec in arr[i:i+win]]))
    return ans

In [32]:
win = 16
raw_x = getMat(raw_x , win)
raw_y = raw_y[win-1:]
print "\n",np.shape(raw_x), np.shape(raw_y)

===================================================(1268079, 16, 16) (1268079,)


### Parse data into train and test

In [34]:
N_train = len(raw_y)*7/10
N_test = len(raw_y) - N_train
x_train = raw_x[:N_train]
x_test = raw_x[N_train:]
y_train = raw_y[:N_train]
y_test = raw_y[N_train:]

In [37]:
def comp_cls_wts(y):
    dic = {}
    for x in set(y):
        dic[x] = np.sqrt(len(y))/np.sqrt(list(y).count(x))
    return dic
cls_wts = comp_cls_wts(y_train)
cls_wts

{0: 1.0566832417372822, 1: 4.3662736626375809, 2: 4.3872418708304437}

In [39]:
y_train = np_utils.to_categorical(y_train, 3)
y_test = np_utils.to_categorical(y_test, 3)

### Construct model

In [42]:
n_cls = 3
model = Sequential()
model.add(Conv2D(32, kernel_size = (4, 4), input_shape = (win, win, 1), padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Conv2D(64, kernel_size = (3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(n_cls, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 16, 16, 32)        544       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
activation_4 (Activation)    (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
activation_5 (Activation)    (None, 4, 4, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 4, 4, 64)          0         
__________

### Compile the model

In [43]:
optimizer = SGD()
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

### Train the model

In [ ]:
res = model.fit(x_train, y_train, batch_size=128, epochs = 6, verbose = 1, validation_data=(x_test, y_test), 
                class_weight = cls_wts)